# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#Discount-Rates"><span class="toc-item-num">3 - </span>Discount Rates</a></div><div class="lev1"><a href="#Technical-Data-Corp.-valuation"><span class="toc-item-num">4 - </span>Technical Data Corp. valuation</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas

# Set _CONSTANTS_

In [2]:
# U.S. Corporate Tax rate
US_CORP_TAX_RATE = .48

# Risk-Free Rate
RISK_FREE_RATE = .11

# Public Market Risk Premium
PUB_MKT_RISK_PREM = .08

# CAPM Betas
PUB_MKT_BETA_FOR_FINANCIAL_SECTOR = 1.   # assumed in line with overall equity market
PEVC_BETA_FOR_PRO_FORMA_PERIOD = 2.

# Discount Rates

Discount Rate for the _pro forma_ start-up period:

In [3]:
pro_forma_period_discount_rate = \
    RISK_FREE_RATE + PEVC_BETA_FOR_PRO_FORMA_PERIOD * PUB_MKT_RISK_PREM
    
pro_forma_period_discount_rate

0.27

Long-term Discount Rate:

In [4]:
long_term_discount_rate = \
    RISK_FREE_RATE + PUB_MKT_BETA_FOR_FINANCIAL_SECTOR * PUB_MKT_RISK_PREM
    
long_term_discount_rate

0.19

# Technical Data Corp. valuation

In [5]:
# initiate Valuation Calcs data frame
val_calcs_df = pandas.DataFrame(index=[1981, 1982, 1983])

Enter provided Revenue, Operating Expenses & Net Operating Profit projections:

In [6]:
val_calcs_df['Rev'] = \
    [288000, 799350, 1181700]
val_calcs_df['OpEx'] = \
    [-242774, -308991, -410010]

# original Net Operating Profit in the pro forma statements
val_calcs_df['NetOpPL'] = \
    val_calcs_df.Rev + \
    val_calcs_df.OpEx

val_calcs_df.T

,1981,1982,1983
Rev,288000,799350,1181700
OpEx,-242774,-308991,-410010
NetOpPL,45226,490359,771690


Take into account the proposed compensation to new executives to arrive at projected EBIT:

In [7]:
# calculate EBIT after compensation for additional executives
val_calcs_df['new_exec_comp'] = \
    [-2500, -75000, -140000]
val_calcs_df['EBIT'] = \
    val_calcs_df.NetOpPL + \
    val_calcs_df.new_exec_comp

val_calcs_df.T

,1981,1982,1983
Rev,288000,799350,1181700
OpEx,-242774,-308991,-410010
NetOpPL,45226,490359,771690
new_exec_comp,-2500,-75000,-140000
EBIT,42726,415359,631690


Calculate EBIAT by using the U.S. corporate tax rate:

In [8]:
# calculate EBIAT
val_calcs_df['EBIAT'] = \
    (1 - US_CORP_TAX_RATE) * val_calcs_df.EBIT
    
val_calcs_df.T

,1981,1982,1983
Rev,288000.00,799350.00,1181700.0
OpEx,-242774.00,-308991.00,-410010.0
NetOpPL,45226.00,490359.00,771690.0
new_exec_comp,-2500.00,-75000.00,-140000.0
EBIT,42726.00,415359.00,631690.0
EBIAT,22217.52,215986.68,328478.8


As the case provides no details on projected investments / divestments and changes in working capital, we assume the FCF before Terminal Value to equal the estimated EBIAT:

In [9]:
val_calcs_df['FCF_before_TV'] = val_calcs_df.EBIAT

val_calcs_df.T

,1981,1982,1983
Rev,288000.00,799350.00,1181700.0
OpEx,-242774.00,-308991.00,-410010.0
NetOpPL,45226.00,490359.00,771690.0
new_exec_comp,-2500.00,-75000.00,-140000.0
EBIT,42726.00,415359.00,631690.0
EBIAT,22217.52,215986.68,328478.8
FCF_before_TV,22217.52,215986.68,328478.8


Let's now estimate the Terminal Value using an assumed long-term growth rate:

In [10]:
terminal_year = 1983
long_term_growth_rate = .08

val_calcs_df['TerminalVal'] = 0
val_calcs_df.loc[terminal_year, 'TerminalVal'] = \
    (1 + long_term_growth_rate) * \
    val_calcs_df.FCF_before_TV.loc[terminal_year] / \
    (long_term_discount_rate - long_term_growth_rate)
    
val_calcs_df.T

,1981,1982,1983
Rev,288000.00,799350.00,1181700.000000
OpEx,-242774.00,-308991.00,-410010.000000
NetOpPL,45226.00,490359.00,771690.000000
new_exec_comp,-2500.00,-75000.00,-140000.000000
EBIT,42726.00,415359.00,631690.000000
EBIAT,22217.52,215986.68,328478.800000
FCF_before_TV,22217.52,215986.68,328478.800000
TerminalVal,0.00,0.00,3225064.581818


Valuation of the company:

In [11]:
val_of_cash_flows_in_pro_forma_period = \
    numpy.npv(
        rate=pro_forma_period_discount_rate,
        values=[0] + val_calcs_df.FCF_before_TV.tolist())
    
val_of_terminal_val = \
    numpy.npv(
        rate=long_term_discount_rate,
        values=[0] + val_calcs_df.TerminalVal.tolist())
    
val = val_of_cash_flows_in_pro_forma_period + val_of_terminal_val

'{:,}'.format(val)

'2,225,570.4941'